In [10]:
import numpy as np
import matplotlib.pyplot as plt
import random
import glob
import cv2
import datetime
import keras
import tensorflow as tf
import os
import shutil
import sys
import time
import json
import argparse
import warnings
import seaborn as sns

In [11]:
# Data preprocessing
from keras.preprocessing.image import ImageDataGenerator  # For image data augmentation
from keras.applications.inception_v3 import preprocess_input  # For preprocessing input images

# Model architecture
from keras import Sequential  # For building sequential models
from keras.models import load_model  # For loading pre-trained models
from keras.layers import Dense, GlobalAvgPool2D as GAP, Dropout  # For defining model layers

# Training callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping  # For training callbacks

# Pre-trained models
from tensorflow.keras.applications import InceptionV3, Xception, ResNet152V2  # For using pre-trained models


In [38]:
#Set path
image_dataset = '../final_project/dataset/dataset/'
class_names_path = '../final_project/translation.json'

# Load class names from JSON file
class_names = json.load(open(class_names_path))
class_names = {v for k, v in class_names.items()}
class_names = sorted(class_names)
print("Class name list: ",class_names)

num_classes = len(class_names)
print("Total species in dataset: ",num_classes)

Class name list:  ['African Bush Elephant', 'African Lionfish', 'African Penguin', 'African Spurred Tortoise', 'Altamira Oriole', 'American Bison', 'American Cockroach', 'American Flamingo', 'American Marten', 'American Robin', 'American white ibis', 'Andean Condor', 'Ankylosaurus', 'Ant', 'Bald Eagle', 'Baltimore Oriole', 'Bee Hummingbird', 'Beluga', 'Bighorn Sheep', 'Black Rat', 'Black-capped Chickadee', 'Blue Jay', 'Blue Whale', 'Boto', 'Brown-throated Three-toed Sloth', 'Bullock Mountains False Toad', 'Canada Goose', 'Carolina Wren', 'Cat', 'Cheetah', 'Chicken', 'Coelacanth', 'Common Bottlenose Dolphin', 'Common Eland', 'Common House Fly', 'Common Lionfish', 'Common Ostrich', 'Corn Snake', 'Cougar', 'Crested Auklet', 'Crested Giant Gecko', 'Crocodile', "Dead Man's Fingers", 'Diplodocus', 'Domestic Cow', 'Domestic Dog', 'Dugong', 'Eastern Copperhead', 'Eastern Gray Squirrel', 'Eastern Kingbird', 'Eastern Ratsnake', 'Eastern Tiger Swallowtail', 'Emperor Penguin', 'Fossa', 'Gaur', 'Gh

In [43]:
# Get the number of samples in each class
class_names_dir = []
for sub_dir in os.listdir(image_dataset):
    sub_name = os.path.join(image_dataset, sub_dir)
    if os.path.isdir(sub_name):
        num_items = len(os.listdir(sub_name))
        class_names_dir.append(num_items)

#size of each sub-class
class_names_size = dict(zip(class_names, class_names_dir))
print("Class name and size: ", class_names_size)

Class name and size:  {'African Bush Elephant': 33, 'African Lionfish': 50, 'African Penguin': 34, 'African Spurred Tortoise': 50, 'Altamira Oriole': 31, 'American Bison': 33, 'American Cockroach': 34, 'American Flamingo': 50, 'American Marten': 32, 'American Robin': 50, 'American white ibis': 30, 'Andean Condor': 49, 'Ankylosaurus': 50, 'Ant': 49, 'Bald Eagle': 50, 'Baltimore Oriole': 31, 'Bee Hummingbird': 50, 'Beluga': 34, 'Bighorn Sheep': 50, 'Black Rat': 31, 'Black-capped Chickadee': 50, 'Blue Jay': 50, 'Blue Whale': 32, 'Boto': 31, 'Brown-throated Three-toed Sloth': 32, 'Bullock Mountains False Toad': 50, 'Canada Goose': 50, 'Carolina Wren': 60, 'Cat': 50, 'Cheetah': 50, 'Chicken': 32, 'Coelacanth': 60, 'Common Bottlenose Dolphin': 60, 'Common Eland': 50, 'Common House Fly': 50, 'Common Lionfish': 30, 'Common Ostrich': 50, 'Corn Snake': 36, 'Cougar': 60, 'Crested Auklet': 31, 'Crested Giant Gecko': 34, 'Crocodile': 50, "Dead Man's Fingers": 50, 'Diplodocus': 50, 'Domestic Cow': 3